#FedMD (I): training clients on public data

In [ ]:
# Clone GitHub repository

import os

if not os.path.isdir('content/aml_project'):
  !git clone https://github.com/s295103/aml_project.git
  %cd /content/aml_project
else:
  if os.getcwd() != "/content/aml_project":
    %cd /content/aml_project/
  !git pull origin

import torch
from utils import training, cifar_processing, load_model
from fedmd import create_clients, save_clients

ROOT = "/content"

# Load and preprocess CIFAR10
train_set, _, test_set = cifar_processing(False, 0, ROOT)

# Make clients
num_classes = len(train_set.classes)
clients = create_clients(num_classes)


##Train the clients on CIFAR10 until convergence

In [ ]:
# Make results folder
BL_PATH = f"{ROOT}/baselines"
if not os.path.isdir(BL_PATH):
  !mkdir $BL_PATH

# Training function keyword arguments
kwargs = dict(
    batch_size = 128,
    device = "cuda" if torch.cuda.is_available() else "cpu",
    num_workers = 8,
    path = BL_PATH,
    lr = 1e-1,
    momentum = 0.9,
    weight_decay = 1e-4,
    num_epochs = 0,
    resume_file = None,
    test_freq = 5
  )

# Train
for name, client in clients.items():
  print(f"Training client {name} on CIFAR10")
  test_acc = training(name, client.model, train_set, test_set, **kwargs)
  print(f"Client {name} final test accuracy: {100*test_acc:.1f} %")

In [4]:
# Load best model onto clients and serialize them
for name, client in clients.items():
    client_data = load_model(f"{BL_PATH}/{name}_best_model.pth")
    client.model.load_state_dict(client_data["weights"])
save_clients(clients, BL_PATH)

ImportError: attempted relative import with no known parent package